In [18]:
# 基于ECFP的数据包处理
# 最后更新：2020.05.07 戴以恒
# 参数及说明：
# 输入文件：包含两列，第一列标题为'smiles'，第二列标题任意，为标签值
# 输出文件：一共6个：X，y，特征列标题Title，无标题的SMILES，两个整合的数据集
INPUT_NAME = "CuData_94.csv"
RADIUS = 3                                # 整数，设置ECFP提取特征的半径

# V1.5加入'特征列填充率保留模块'：
FEATURE_RESERVE = 0.011                   # 浮点数，特征矩阵内非零数据的比例大于等于这个一般保留阈值，这个特征就会被保留
EXACT_PIECE_RESERVE = True                # 布尔值，表示是否开启使用特定基团保留片段的功能
SMILES_PATT = ['CO']                      # 字符串列表，每个元素都是一段SMILES值(其实用的是SMART，这样可以匹配类似'ccc'的芳环残片)，用于在保留时作为匹配模式(仅在EXACT_PIECE_RESERVE为True时生效)
EXACT_PIECE_RESERVE_THRESHOLD = 0.011     # 浮点数，针对特定基团的阈值，和上面的类似，若想要全部保留含有某特定基团的特征，此项请用0.0(仅在EXACT_PIECE_RESERVE为True时生效)
# 假如只想保留含有特定基团的片段，可以把FEATURE_RESERVE设为1.0，而EXACT_PIECE_RESERVE设为0.0，因为特定基团筛选的优先级高于一般保留阈值的筛选

# V2.0为针对'CO'的处理而加入，V5.0修改为更具有普遍性的'布尔值转化模块'：
# 作用：将一个max值为n的特征列转化为n或(n+1)列布尔值特征列，分别表示拥有(0),1,2,3,...,n个特征片段
# 注意：若进行全部数据的转化以求获得一个只由{0, 1}构成的特征矩阵，不应和加入RDKit描述符的功能一起使用
# V5.1：添加了自动删除空特征列的功能
# V6.0：整合了两种转化为Bool值的模式，即a版(n列)与b版(n+1列)
TURN_TO_BOOL = False                      # 布尔值，作为布尔值转化模块的总开关使用
ALL_TURN_TO_BOOL = True                   # 布尔值，若为True，则将所有非{0, 1}整数数据全部转化为多个布尔值特征列，最后的特征值矩阵将完全由{0, 1}构成
BOOL_PATT_LIST = []                       # 字符串列表，仅在ALL_TURN_TO_BOOL为False时生效，表示将列表中指定的基团转变为布尔值特征内(注意：必须特征列的标题和这里面的字符串一样才进行转换操作)
BOOL_TURN_MODE = 'a'                      # 字符，'a'或'b'，代表两种布尔值转化的模式，a为n列，b为n+1列

# V3.0加入'RDKit描述符合并模块'：
INCLUDE_DESC = False                      # 布尔值，表示是否需要计算并加入RDKit的描述符

# V4.0加入'片段原子数量调控模块'：
ATOM_COUNT_CONTROL = True                 # 布尔值，作为“原子数量调控模块”的总开关
SMARTS_MIN_LENGTH = 2                     # 整数值，表示特征SMARTS内非氢原子的最小数目，包含非氢原子数目大于等于该数目的基团片段才会被保留，设为0就会跳过这一步骤
SMARTS_MAX_LENGTH = 20                    # 整数值，表示特征SMARTS内非氢原子的最小数目，包含非氢原子数目小于等于该数目的基团片段才会被保留，若不想在最大值上设限，可设为5*RADIUS或更大的数值
ATOM_COUNT_CONTROL_OMIT_PATTERN = []      # 字符串列表，表示若特征片段内含有此列表中的SMARTS片段，此列将不受“原子数量调控模块”的调控

# V4.1加入：新输出一个带Title和Value的完整的数据矩阵

In [19]:
import numpy as np
import matplotlib.pyplot as plt

In [20]:
from deepchem.utils.save import load_from_disk
dataset_file= INPUT_NAME
dataset = load_from_disk(dataset_file)
# 数据集默认第一列是SMILES，第二列是标签值，放入数据前请检查好
print("Columns of dataset: %s" % str(dataset.columns.values))
print("Number of examples in dataset: %s" % str(dataset.shape[0]))

Columns of dataset: ['smiles' 'FEC2+(20%)']
Number of examples in dataset: 94


In [21]:
# 定义‘特征器‘’(Featurizer)来将SMILES转化为某些特征
import deepchem as dc
# featurizer_fix = dc.feat.CircularFingerprint(size=1024)
featurizer_sparse = dc.feat.CircularFingerprint(size=1024, sparse=True, smiles=True, radius=RADIUS)
if INCLUDE_DESC:
    featurizer_desc = dc.feat.RDKitDescriptors()

In [22]:
# 对SMILES进行特征化，输出为哈希值以及SMILES块
# 因为hash值与smiles块一一对应，我们只需挑出unique的SMILES/hash作为特征即可
loader_sparse = dc.data.CSVLoader(
      tasks=[dataset.columns.values[1]], smiles_field=dataset.columns.values[0],
      featurizer=featurizer_sparse)
dataset_sparse = loader_sparse.featurize(dataset_file)
print('Sparse:')
print(dataset_sparse.get_shape())
# 此时的特征“矩阵”应当为一个大的字典
print(dataset_sparse.get_shard(0)[0].shape)

Loading raw samples now.
shard_size: 8192
About to start loading CSV from CuData_94.csv
Loading shard 1 of size 8192.
Featurizing sample 0
TIMING: featurizing shard 0 took 0.160 s
TIMING: dataset construction took 0.171 s
Loading dataset from disk.
Sparse:
((94,), (94, 1), (94, 1), (94,))
(94,)


In [23]:
# 取出所有的分子，这一步得到一个94长度的列表，每个元素为1个字典，字典中为hash:{'smiles':'str', 'count':, int}
fps_sparse = list(dataset_sparse.get_shard(0)[0])
# print(fps_sparse)
# print(len(fps_sparse))
# # 这一步得到一个所有hash值的列表hash_list
# hash_list = []
# for i in range(94):
#     sparse_list = list(fps_sparse[i])
#     for j in range(len(sparse_list)):
#         hash_list.append(sparse_list[j])
# print(len(hash_list))
# # 这一步得到所有特异的hash值
# hash_list_unique = np.unique(hash_list)
# print(len(hash_list_unique))

In [24]:
smiles = []
# count = []
for i in range(dataset_sparse.get_shard(0)[0].shape[0]):
    # 这一步得到的依然是1个字典
    fps_dict = fps_sparse[i]
    for hash_value, sub_dict in fps_dict.items():
        # 这一步把所有的smiles和count挂进列表，count会影响某种特征的权重，也有必要记录下来
        smiles.append(sub_dict['smiles'])
        # count.append(sub_dict['count'])
print('unique hash:', len(smiles))
# 这一步得到所有特异的smiles，发现1个smiles可能对应多个hash
smiles_unique = np.unique(smiles)
print('unique pieces', len(smiles_unique))
# 目标是把特征矩阵缩到smiles_unique的长度，这样每一列代表一种特征，特征值即为count的权重
# 如果这样，可以选择跳过hash，直接统计特征

unique hash: 2205
unique pieces 817


In [25]:
title = []
data = np.zeros((dataset_sparse.get_shard(0)[0].shape[0], len(smiles_unique)-1))
idx = 0
for i in range(dataset_sparse.get_shard(0)[0].shape[0]):
    # 这一步得到的依然是1个字典
    fps_dict = fps_sparse[i]
    # 数据提炼去重：
    for hash_value, sub_dict in fps_dict.items():
        SMILES = sub_dict['smiles']
        COUNT = sub_dict['count']
        # 跳过'这个空SMILES
        if SMILES=='':
            continue
        if SMILES not in title:
            title.append(SMILES)
            data[i][idx] += COUNT
            idx +=1
        else:
            data[i][title.index(SMILES)] +=COUNT
print(idx)
print(len(title))
print(data.shape)
data = data.astype(int)

816
816
(94, 816)


In [26]:
from rdkit import Chem
# 对于过于稀疏的特征列，设置FEATURE_RESERVE特征保留阈值进行筛选，列填充比例大于此的才会被保留
# 同时还要保留含有特定基团的片段，此操作的优先级高于按保留阈值筛选
delete = []
for i in range(data.shape[1]):
    filled_rate = sum(data[:, i]!=0)/data.shape[0]
    if filled_rate<FEATURE_RESERVE:
        if (EXACT_PIECE_RESERVE):                               # 这一块是在挑出含有指定集团的片段
            m = Chem.MolFromSmarts(title[i])
            for patt_temp in SMILES_PATT:
                patt = Chem.MolFromSmarts(patt_temp)
                flag = m.HasSubstructMatch(patt)
                if flag:
                    break
            if flag and (filled_rate>=EXACT_PIECE_RESERVE_THRESHOLD):
                continue
        delete.append(i)
data = np.delete(data, delete, axis=1)
title = np.array(title).reshape(1, len(title))
title = np.delete(title, delete, axis=1)
title = title.flatten().tolist()
print('After delete sparse coloumns: length of title:', len(title))
print('After delete sparse coloumns: shape of data:', data.shape)

After delete sparse coloumns: length of title: 163
After delete sparse coloumns: shape of data: (94, 163)


In [27]:
# 原子数量调控模块：
if ATOM_COUNT_CONTROL:
    delete = []
    for i in range(data.shape[1]):
        flag = False
        for patt_temp in ATOM_COUNT_CONTROL_OMIT_PATTERN:
            patt = Chem.MolFromSmarts(patt_temp)
            flag = m.HasSubstructMatch(patt)
            if flag:
                break
        if flag:
            continue
        m = Chem.MolFromSmarts(title[i])
        atom_count = len(m.GetAtoms())
        if atom_count<SMARTS_MIN_LENGTH or atom_count>SMARTS_MAX_LENGTH:
            delete.append(i)
    data = np.delete(data, delete, axis=1)
    title = np.array(title).reshape(1, len(title))
    title = np.delete(title, delete, axis=1)
    title = title.flatten().tolist()
    print('After atom-count control: length of title:', len(title))
    print('After atom-count control: shape of data:', data.shape)

After atom-count control: length of title: 163
After atom-count control: shape of data: (94, 163)


In [28]:
# 这一块里，会删除某一指定的特征列，将其转化为表示含有0,1,...,n个基团的布尔表示列，其中n为所有样本中含基团的最大值，最后会自动把空列删除
if TURN_TO_BOOL:
    if ALL_TURN_TO_BOOL:
        BOOL_PATT_LIST = title
    for bool_patt in BOOL_PATT_LIST:
        print('Processing feature: ', bool_patt)
        print('title length(before turn to bool):', len(title))
        print('data shape(before turn to bool):', data.shape)
        index_t = title.index(bool_patt)
        print('Target conlumn index:', index_t)
        feature_max = int(max(data[:, index_t]))
        print('max number of target patt:', feature_max)
        # 制作新的bool数据数组
        if BOOL_TURN_MODE=='a':
            matrix_bool = np.zeros((data.shape[0], feature_max))
        else:
            matrix_bool = np.zeros((data.shape[0], feature_max+1))
        for i in range(data.shape[0]):
            if BOOL_TURN_MODE=='a':
                if data[i, index_t]>=1:
                    matrix_bool[i, int(data[i, index_t]-1)] = 1
            else:
                matrix_bool[i, int(data[i, index_t])] = 1
        # 删除对应的数据列和标题列
        data = np.delete(data, [index_t], axis=1)
        title = np.array(title).reshape(1, len(title))
        title = np.delete(title, [index_t], axis=1)
        title = title.flatten().tolist()
        # 制作额外的标题
        new_title = []
        if BOOL_TURN_MODE=='a':
            for i in range(feature_max):
                new_title.append('With_'+str(i+1)+'_'+bool_patt)
        else:
            for i in range(feature_max+1):
                new_title.append('With_'+str(i)+'_'+bool_patt)
        data = np.hstack((data, matrix_bool))
        title = title+new_title
        print('title length(after turn to bool):', len(title))
        print('data shape(after turn to bool):', data.shape, '\n')
    # 将空列自动删除，V5.1加入
    delete = []
    for i in range(data.shape[1]):
        if sum(data[:, i])==0:
            delete.append(i)
    print(data.shape)
    print(len(title))
    data = np.delete(data, delete, axis=1)
    title = np.array(title).reshape(1, len(title))
    title = np.delete(title, delete, axis=1)
    title = title.flatten().tolist()
    print(data.shape)
    print(len(title))

In [29]:
# 数据集的整理
print(data.shape)
print(len(title))
X = data
y = dataset_sparse.get_shard(0)[1]
title = np.array(title).reshape(X.shape[1], )
smiles_out = dataset_sparse.get_shard(0)[3]

(94, 163)
163


In [30]:
# 如果需要包含描述符，这会在这里进行计算和矩阵的合并
if INCLUDE_DESC:
    from rdkit.Chem import Descriptors
    loader_desc = dc.data.CSVLoader(
        tasks=[dataset.columns.values[1]], smiles_field=dataset.columns.values[0],
        featurizer=featurizer_desc)
    dataset_desc = loader_desc.featurize(dataset_file)
    print('Desc:')
    Desc_list = dc.feat.RDKitDescriptors.allowedDescriptors
    print(dataset_desc.get_shard(0)[0].shape)
    allow = []
    for i in Descriptors.descList:
        if i[0] in Desc_list:
            allow.append(i[0])
    print(len(allow))
    X_desc = dataset_desc.get_shard(0)[0]
    Desc_list = np.array(allow).reshape(1, len(allow))
    delete = []
    for i in range(X_desc.shape[1]):
        if max(X_desc[:, i])==min(X_desc[:, i]):
            delete.append(i)
    X_desc = np.delete(X_desc, delete, axis=1)
    Desc_list = np.delete(Desc_list, delete, axis=1).flatten().tolist()
    X = np.hstack((X, X_desc))
    title = title.flatten().tolist()
    title = title+Desc_list
    title = np.array(title).reshape(len(title), )

In [31]:
# 创建一个目录来保存生成的数据包
import os
import time
from pathlib import Path
if INCLUDE_DESC:
    DIR = time.strftime("%Y%m%d_%H%M%S", time.localtime())+'_'+str(X.shape[0])+'_'+str(data.shape[1])+'+'+str(X_desc.shape[1])
else:
    DIR = time.strftime("%Y%m%d_%H%M%S", time.localtime())+'_'+str(X.shape[0])+'_'+str(X.shape[1])
os.mkdir(DIR)

In [32]:
print(title.shape, X.shape)
# 保存一个带Title但是不带标签值的数据包
out = np.vstack((title, X))
if INCLUDE_DESC:
    OUT_NAME0 = 'Data_'+str(data.shape[0])+'_'+str(data.shape[1])+'+'+str(X_desc.shape[1])+'_withTitle_Nolabel.csv'
else:
    OUT_NAME0 = 'Data_'+str(data.shape[0])+'_'+str(data.shape[1])+'_withTitle_Nolabel.csv'
dir_out0 = Path('.', DIR, OUT_NAME0)
np.savetxt(dir_out0, out, fmt='%s', delimiter=',')

(163,) (94, 163)


In [33]:
# 输出其它数据包
print('X: ', X.shape, '    y: ', y.shape, '    title:', title.shape, '    smiles_list:', smiles_out.shape)
if INCLUDE_DESC:
    OUT_NAME1 = 'Features_'+str(data.shape[0])+'_'+str(data.shape[1])+'+'+str(X_desc.shape[1])+'.csv'
else:
    OUT_NAME1 = 'Features_'+str(data.shape[0])+'_'+str(data.shape[1])+'.csv'
dir_out1 = Path('.', DIR, OUT_NAME1)
OUT_NAME2 = 'Values_'+str(data.shape[0])+'.csv'
dir_out2 = Path('.', DIR, OUT_NAME2)
if INCLUDE_DESC:
    OUT_NAME3 = 'Title_'+str(data.shape[1])+'+'+str(X_desc.shape[1])+'.csv'
else:
    OUT_NAME3 = 'Title_'+str(data.shape[1])+'.csv'
dir_out3 = Path('.', DIR, OUT_NAME3)
OUT_NAME4 = 'SMILES_'+str(data.shape[0])+'.csv'
dir_out4 = Path('.', DIR, OUT_NAME4)
np.savetxt(dir_out1, X, fmt='%s', delimiter=',')
np.savetxt(dir_out2, y, fmt='%s')
np.savetxt(dir_out3, title, fmt='%s')
np.savetxt(dir_out4, smiles_out, fmt='%s')

X:  (94, 163)     y:  (94, 1)     title: (163,)     smiles_list: (94,)


In [34]:
# 输出一个带Title和Label的无SMILES完整数据集
title = title.flatten().tolist()
title = title+['value']
title = np.array(title).reshape(len(title), )
print(X.shape, y.shape, title.shape)
out = np.vstack((title, np.hstack((X, y))))
print(out.shape)
if INCLUDE_DESC:
    OUT_NAME5 = 'Data_'+str(data.shape[0])+'_'+str(data.shape[1])+'+'+str(X_desc.shape[1])+'_withTitle_withLabel.csv'
else:
    OUT_NAME5 = 'Data_'+str(data.shape[0])+'_'+str(data.shape[1])+'_withTitle_withLabel.csv'
dir_out5 = Path('.', DIR, OUT_NAME5)
np.savetxt(dir_out5, out, fmt='%s', delimiter=',')

(94, 163) (94, 1) (164,)
(95, 164)
